In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [10]:
df= pd.read_excel('test1.xlsx')
question_list=[]
is_hots_list=[]
q_name='question'
h_name="is_hots"

In [11]:
vocab_size = 1000
embedding_dim = 16
max_length = 20
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
num_epochs=150
train_size=180

In [12]:
question_list.extend(df[q_name].tolist())
is_hots_list.extend(df[h_name].tolist())

training_data=question_list[:train_size]
training_data_hots =is_hots_list[:train_size]
testing_data=question_list[train_size:]
testing_data_hots=is_hots_list[train_size:]

tokenizer=Tokenizer(num_words=20, oov_token="<OOV>")
tokenizer.fit_on_texts(training_data)
word_index=tokenizer.word_index

padded = np.array(pad_sequences(tokenizer.texts_to_sequences(training_data), padding=padding_type, truncating=trunc_type, maxlen=max_length))
sequence = np.array(pad_sequences(tokenizer.texts_to_sequences(training_data), padding=padding_type, truncating=trunc_type, maxlen=max_length))  # Use padded here

# Same for testing data
testpadded = np.array(pad_sequences(tokenizer.texts_to_sequences(testing_data), padding=padding_type, truncating=trunc_type, maxlen=max_length))
testsequence = testpadded  

is_hots_train = np.array(training_data_hots) # Convert list to NumPy array (correct labels)
is_hots_test = np.array(testing_data_hots)

In [13]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
print("Model output shape:", model.layers[-1].output_shape)  # Should be (None, 1)
print("Labels shape:", sequence.shape)  # Should be (None, 1) for binary classification

Model output shape: (None, 1)
Labels shape: (152, 20)


In [15]:
# #troubleshoothing
import tensorflow as tf

# 1. Verify Output Shape
preds = model.predict(padded)
print("Predictions shape:", preds.shape)  # Should be (None, 1)

# 2. Check Label Shape
print("Labels shape:", sequence.shape)   # Should be (None, 20)

# 3. Examine Loss Function
print("Loss function:", model.loss)

# 4. Debug with Print Statements (example)
print("Model output shape:", model.layers[-1].output_shape)
# print("True label shape in loss calculation:", tf.shape(sequence))


5/5 [==============================] - 0s 5ms/step
Predictions shape: (152, 1)
Labels shape: (152, 20)
Loss function: binary_crossentropy
Model output shape: (None, 1)


In [16]:
history = model.fit(padded, is_hots_train, epochs=num_epochs,
                    validation_data=(testpadded, is_hots_test), verbose=2)

Epoch 1/150
5/5 - 2s - loss: 0.6914 - accuracy: 0.6382 - val_loss: 0.6850 - val_accuracy: 0.7843 - 2s/epoch - 340ms/step
Epoch 2/150
5/5 - 0s - loss: 0.6872 - accuracy: 0.6711 - val_loss: 0.6784 - val_accuracy: 0.7843 - 82ms/epoch - 16ms/step
Epoch 3/150
5/5 - 0s - loss: 0.6839 - accuracy: 0.6711 - val_loss: 0.6714 - val_accuracy: 0.7843 - 41ms/epoch - 8ms/step
Epoch 4/150
5/5 - 0s - loss: 0.6801 - accuracy: 0.6711 - val_loss: 0.6646 - val_accuracy: 0.7843 - 38ms/epoch - 8ms/step
Epoch 5/150
5/5 - 0s - loss: 0.6763 - accuracy: 0.6711 - val_loss: 0.6578 - val_accuracy: 0.7843 - 41ms/epoch - 8ms/step
Epoch 6/150
5/5 - 0s - loss: 0.6720 - accuracy: 0.6711 - val_loss: 0.6509 - val_accuracy: 0.7843 - 39ms/epoch - 8ms/step
Epoch 7/150
5/5 - 0s - loss: 0.6686 - accuracy: 0.6711 - val_loss: 0.6433 - val_accuracy: 0.7843 - 40ms/epoch - 8ms/step
Epoch 8/150
5/5 - 0s - loss: 0.6645 - accuracy: 0.6711 - val_loss: 0.6353 - val_accuracy: 0.7843 - 40ms/epoch - 8ms/step
Epoch 9/150
5/5 - 0s - loss: 0.